## Set up a duckweed genotypes x media growth assay

### Pre-requisites to use this script
1. Precise plate and well positions for your machine defined in Plate_positions.py in the 'utils' subdirectory. 
2. All the parameters defined in the cell 'User paramters'. 
3. Sufficient 24-well plates, sterile media and duckweed plants available. 
4. Jubilee machine set up with Media-dispensing syringe tool (50 mL), Duckweed transfer syringe tool and lab automation bedplate. 


In [1]:
#Importing python libraries downloaded from the internet
import random
import pandas as pd
import os
import json
import time
import math

In [2]:
#Importing python libraries from local files. 
import utils.DuckbotExptSetupUtils as exp
from utils.MachineUtils import *
from utils.CameraUtils import *
import utils.PlatePositionUtils as pp 

In [3]:
port = '/dev/ttyACM0'
m = MachineCommunication(port)

## 1. Define user parameters

### Labware config 
All should be in arbitrary Jubilee motor units. Find this manually on the machine. They shouldn't need to be updated until you start using a new reservoir. And if that's the case you could create a labware library and then define the media_reservoir variable as one of the objects in your library. 

In [4]:
#X and Y should get the machine to the center of the reservoir. 
media_reservoir = { 'x' : 75, 'y' : 241,}

#Tool positions
media_syringe = 2 #What jubilee tool position did you define this as?
duckweed_syringe = 3

#Volume calculations
dispenses_per_syringe_fill = 20
dispense_mL = 1.5 #In mL
vol_conversion = 3.9 #One mL is 3.9 units. 
dispense_offset = dispense_mL * vol_conversion

#Z-positions for different actions
z_dict = {"zero": 0, #Note that zeros
          "aspirate" : -46.5, #Measure this for yourself and your labware/toolheads.
          "dispense" : -21.5
         }

### Define experimental variables, and desired file names and save locations. 

In [5]:

#1. DEFINE EXPERIMENTAL VARIABLES
genotypes = ["Sp7498"] # Replace with names for unique duckweed genotypes
media = ["Mock", "Salt"] # Replace with names for unique media
reps  = 12 # Replace with your desired number of replicates for each duckweed/media combination. 

#2. DEFINE FILE LOCATION AND NAME
expt_setup_parent_dir = os.getcwd() # Default uses current working directory but you can replace with your own choice. 
expt_setup_dir = "LabDemo" # Name of the folder to hold experiment data and metadata including the setup file
expt_setup_filename = "LabDemo.json" #Name for the experiment setup file (Metadata)

expt_setup_file_path = os.path.join(expt_setup_parent_dir, expt_setup_dir)
print(expt_setup_file_path)

if not os.path.exists(expt_setup_file_path):
    os.mkdir(expt_setup_file_path)     

/home/pi/duckbot/notebooks/LabDemo


## 2. Create dataframe with experiment metadata

In [6]:
# Creates master list of sample info, shuffles and then assigns to plates and wells. 
master_expt_list = []

for g in genotypes:
    for m in media:
        for x in range(reps):
             master_expt_list.append({"genotype": g, "media": m, "condition_replicate": x + 1})


random.shuffle(master_expt_list)
master_expt_list = exp.assign_plates_and_wells(master_expt_list)
expt_dict = {"sample_info" : master_expt_list}

In [7]:
#Save experimental set up file
os.chdir(expt_setup_file_path)
with open(expt_setup_filename, 'w') as f:
    json.dump(expt_dict, f)

In [8]:
#Import from file (in case user wants to make any manual edits to the JSON file after creating it)
os.chdir(expt_setup_file_path)
with open(expt_setup_filename) as datafile:
    expt_data = json.load(datafile)

# Turn samples list into a dataframe
sample_data = expt_data["sample_info"]
df = pd.DataFrame(sample_data)
print(df)

   genotype media  condition_replicate    plate_well_id    Plate Well
0    Sp7498  Mock                    7  Plate_1_Well_A1  Plate_1   A1
1    Sp7498  Mock                    1  Plate_1_Well_A2  Plate_1   A2
2    Sp7498  Mock                   10  Plate_1_Well_A3  Plate_1   A3
3    Sp7498  Salt                    3  Plate_1_Well_A4  Plate_1   A4
4    Sp7498  Mock                    6  Plate_1_Well_A5  Plate_1   A5
5    Sp7498  Mock                    9  Plate_1_Well_A6  Plate_1   A6
6    Sp7498  Salt                    4  Plate_1_Well_B1  Plate_1   B1
7    Sp7498  Salt                   10  Plate_1_Well_B2  Plate_1   B2
8    Sp7498  Mock                   12  Plate_1_Well_B3  Plate_1   B3
9    Sp7498  Salt                    9  Plate_1_Well_B4  Plate_1   B4
10   Sp7498  Salt                   12  Plate_1_Well_B5  Plate_1   B5
11   Sp7498  Salt                    2  Plate_1_Well_B6  Plate_1   B6
12   Sp7498  Mock                    3  Plate_1_Well_C1  Plate_1   C1
13   Sp7498  Mock   

## 3. Label 24-well plates and add to machine

#### Label Plates and add plates to machine

In [9]:
num_plates = df.Plate.nunique()
print(num_plates)

print("This experiment requires {} 24-well plate(s)".format(num_plates))
print("----")
lst = list(range(1,num_plates + 1))
for n in lst:
    print("Label a plate with experiment ID or initials and 'plate {}'".format(n))
print("----")
print ("Place the 24-well plate(s) in the jubilee".format(num_plates))
print ("Start at position 1 and fill empty plate slots in order")

1
This experiment requires 1 24-well plate(s)
----
Label a plate with experiment ID or initials and 'plate 1'
----
Place the 24-well plate(s) in the jubilee
Start at position 1 and fill empty plate slots in order


## 2. Dispense media
When prompted insert containers of the relevant sterile media into the input slot on the Jubilee. 

In [10]:
#Retrieve absolute positions of wells from a library and then add those coordinatest to the plate set up dataframe
df = pp.add_well_coords_to_df_from_file(expt_setup_file_path, expt_setup_filename)
print(df)

   genotype media  condition_replicate    plate_well_id    Plate Well      x  \
0    Sp7498  Mock                    7  Plate_1_Well_A1  Plate_1   A1   29.0   
1    Sp7498  Mock                    1  Plate_1_Well_A2  Plate_1   A2   48.0   
2    Sp7498  Mock                   10  Plate_1_Well_A3  Plate_1   A3   67.0   
3    Sp7498  Salt                    3  Plate_1_Well_A4  Plate_1   A4   86.0   
4    Sp7498  Mock                    6  Plate_1_Well_A5  Plate_1   A5  105.0   
5    Sp7498  Mock                    9  Plate_1_Well_A6  Plate_1   A6  124.0   
6    Sp7498  Salt                    4  Plate_1_Well_B1  Plate_1   B1   29.0   
7    Sp7498  Salt                   10  Plate_1_Well_B2  Plate_1   B2   48.0   
8    Sp7498  Mock                   12  Plate_1_Well_B3  Plate_1   B3   67.0   
9    Sp7498  Salt                    9  Plate_1_Well_B4  Plate_1   B4   86.0   
10   Sp7498  Salt                   12  Plate_1_Well_B5  Plate_1   B5  105.0   
11   Sp7498  Salt                    2  

In [11]:
#Reorganizes dataframe to create machine instructions sorted by media-type
media_dicts = pp.pull_list_of_well_coord_dicts_by_dfcolumn(df, 'media')
print(media_dicts)

[{'media': 'Mock', 'well-coords': [[29.0, 175.0], [48.0, 175.0], [67.0, 175.0], [105.0, 175.0], [124.0, 175.0], [67.0, 156.0], [29.0, 137.0], [48.0, 137.0], [67.0, 137.0], [105.0, 137.0], [124.0, 137.0], [48.0, 118.0]]}, {'media': 'Salt', 'well-coords': [[86.0, 175.0], [29.0, 156.0], [48.0, 156.0], [86.0, 156.0], [105.0, 156.0], [124.0, 156.0], [86.0, 137.0], [29.0, 118.0], [67.0, 118.0], [86.0, 118.0], [105.0, 118.0], [124.0, 118.0]]}]


In [12]:
#Pick up syringe toolhead
port = '/dev/ttyACM0'
m = MachineCommunication(port)
m.toolChange(media_syringe)

In [13]:
# Send machine instructions
for media in media_dicts:
    m.moveTo(x=0,y=0,z=0)
    print(f"Please ensure {media['media']} is available in the machine before continuing.")
    print("Change syringe and/or needle if desired")
    while True:
        value = input("Enter 'YES' to confirm that the correct media is in position")
        if value != "YES":
            print("Please confirm")
        else:
            break
    exp.dispense_to_wells(m, media["well-coords"], dispense_offset, dispenses_per_syringe_fill, media_reservoir, z_dict)

Please ensure Mock is available in the machine before continuing.
Change syringe and/or needle if desired
Enter 'YES' to confirm that the correct media is in positionYES
Move to Z = zero
Move to reservoir position
moved to height for aspiration
G1    E81.90 F1000.00
Moved to Z = zero
Hovering over the first well to dispense into
Prepare to dispense
X = 48.0
Y = 175.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 86.0
Y = 175.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 105.0
Y = 175.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 67.0
Y = 156.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 86.0
Y = 156.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 105.0
Y = 156.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 124.0
Y = 156.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 86.0
Y = 137.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 124.0
Y = 137.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 48.0
Y = 118.0
G1    E-5.85 F1000.00
Prepare to dispense
X = 67.0
Y = 118.0
G1    E-5.85 F10

In [12]:
# pop the bed down for easy access to labware
m.moveTo(z=80)

AttributeError: 'str' object has no attribute 'moveTo'

## 5. Transfer duckweed
Place a container filled with fronds of the relevant duckweed type and the machine will attempt to move individual fronds into the relevant wells. After each attempt at filling all wells a camera will take pictures of each well to confirm success and then unsuccessful wells will be reattempted

In [15]:
port = '/dev/ttyACM0'
m = MachineCommunication(port)
m.toolChange(3)

In [ ]:
m.moveTo(x=0,y=0)
print(f"Please ensure the duckweed reservoir and a 22 gauge syringe is available in the machine before continuing.")
while True:
    value = input("Enter 'YES' to confirm that labware is in position")
    if value != "YES":
        print("Please confirm")
    else:
        break

In [18]:
# manually probe the z height of the duckweed reservoir surface
# enter the z value here
# port = '/dev/ttyACM0'
# m = MachineCommunication(port)
duckweed_reservoir = [77, 243, -18.8]
m.moveTo(x=duckweed_reservoir[0], y=duckweed_reservoir[1])
# center (x=77): -17.1
# edge (x=109): -17.5

In [24]:
# accounting for the meniscus
def dist(x1, y1, x2, y2):
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

# def meniscus_compensation(z_center, z_edge, radius):
    


1.4142135623730951

In [32]:
# make sure z value is >0 again after probing water height
m.moveTo(z=80) 

In [33]:
# grab the camera, move the relevant height & position to take pic of duckweed
m.toolChange(1)
m.moveTo(x=duckweed_reservoir[0], y=duckweed_reservoir[1])

# move down more based on surface height
syringe_zero = 22.9 # for blue syringe
syringe_zero = 25 # for green syringe tip
dh = syringe_zero + duckweed_reservoir[2]
m.move(dz=dh)

G1   Z6.20  F6000.00


In [21]:
# load in the relevant calibration files

# with open("/home/pi/autofocus-test/JubileeAutofocus/camera_cal_z_60_tape.json") as f:
with open("/home/pi/autofocus-test/JubileeAutofocus/camera_cal_z_80.json") as f:
    cal = json.load(f)
matrix = np.array(cal['transform'])
size = cal['resolution']
print(matrix)
print(size)

m.transform = matrix
m.img_size = size

[[ -0.7296065   -0.35246752]
 [  0.38097134  -0.71682782]
 [ -0.32375357   0.6160794 ]
 [  1.60644767  83.44887212]
 [-82.50757965   1.39357274]
 [ 75.86369137 238.95022829]]
[1200, 1200]


In [22]:
#focus camera
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

# draw a circle in the center of the frame
center = None
while center is None:
    # the first frame grab is sometimes empty
    ret, frame = cap.read()
    h, w = frame.shape[0:2]
    center = (int(w/2), int(h/2))

while True:
    ret, frame = cap.read()
    target = cv2.circle(frame, center, 5, (0,255,0), -1)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

In [34]:
# manually pick fronds
# alternatively, manually click a frond
f = getFrame()
# saveFrame(f, "/home/pi/Downloads/petri-60.jpg")
pts = selectPoint(f, num_pts=8)


[(569.4276771133058, 450.7948129714597), (767.2657671669648, 521.7179018586205), (715.0066490395833, 333.2117971848511), (393.98635197138196, 549.7138579982892), (537.6989268216813, 549.7138579982892), (563.8284858853721, 822.2078310910647), (743.0026051792519, 710.2240065323898), (841.9216502060814, 622.5033439614278)]


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/usr/lib/python3.7/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/pi/duckbot/.venv/lib/python3.7/site-packages/matplotlib/backends/_backend_tk.py", line 476, in delayed_destroy
    self.window.destroy()
  File "/usr/lib/python3.7/tkinter/__init__.py", line 2061, in destroy
    for c in list(self.children.values()): c.destroy()
  File "/usr/lib/python3.7/tkinter/__init__.py", line 2308, in destroy
    Misc.destroy(self)
  File "/usr/lib/python3.7/tkinter/__init__.py", line 592, in destroy
    self.tk.deletecommand(name)
_tkinter.TclError: can't delete Tcl command


In [35]:
# convert px coord to real coord
def px_to_real(x,y, absolute = False):
        x = (x / m.img_size[0]) - 0.5
        y = (y / m.img_size[1]) - 0.5
        a = 1 if absolute else 0

        return (m.transform.T @ np.array([x**2, y**2, x * y, x, y, a]))

fronds = []
for pt in pts:
    frond_off = px_to_real(pt[0], pt[1])
    frond = [duckweed_reservoir[0] - frond_off[0], duckweed_reservoir[1] - frond_off[1]]
    fronds.append(frond)

In [36]:
# test block to confirm camera-to-machine alignment
idx = 0
fx = fronds[idx][0]
fy = fronds[idx][1]
fx, fy

m.moveTo(x=fx, y=fy)

In [27]:
f=getFrame()
showFrame(f, grid=True)

In [37]:
# switch tools, account for offset
m.toolChange(3)
syringe_off = [-1.23, 5.06] # green
# syringe_off = [0.2, 4.86] # new blue
# syringe_off = [-0.68, 4.82] # old blue
# 73.8, 227.9
print(fronds)

[[66.77773769560457, 245.30865436999318], [71.40330194790998, 231.47462748137985], [58.483638441626425, 235.36397358818505], [73.8414659892143, 257.391951623938], [73.6279149229661, 247.3917282189], [92.31240763909086, 245.2856794524138], [84.39784901635598, 232.9318003249451], [78.25412846194364, 226.1627075712137]]


In [31]:
# testing camera to syringe tool alignment
m.moveTo(m.moveTo(x=fx + syringe_off[0], y=fy + syringe_off[1]))

In [43]:
# aspirate duckweed
# first move to surface
def aspirate():
    m.move(de=5, s=1000) ; # 'prime' syringe
    m.moveTo(z=duckweed_reservoir[2])
    m.dwell(1000)
    m.move(dz=-0.5) # press slightly
    m.dwell(1000)
    m.move(de=30, s=1500) # aspirate!
    m.move(dz=6, s=500) # aspirate!
#     m.move(dz=5, de=50, s=1000) # aspirate!

def aspirate_2():
    m.move(de=5, s=1000) ; # 'prime' syringe
    m.moveTo(z=duckweed_reservoir[2] + 2)
    m.dwell(1000)
    m.move(dz=-3.5, de=5) # press slightly
#     m.dwell(250)
    m.move(de=30, s=1800) # aspirate! #1200 for lemna minor
    m.move(dz=6, s=500, de=5) # aspirate!
#     m.move(dz=5, de=50, s=1000) # aspirate!

def dispense():
    m.dwell(1000)
    m.move(de=-30, s=1400) # dispense
#     m.move(de=-40, s=1800)
    m.dwell(500)

grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    print("""Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing""")
    input() 
    count = 0
    for index,s in sample_df.iterrows():
        #move to plate and well
        frond = fronds[count]
        m.moveTo(m.moveTo(x=frond[0] + syringe_off[0], y=frond[1] + syringe_off[1]))
#         aspirate()
        aspirate()
        m.moveTo(z=6.5)
        well = pp.fetch_well_position(s["Plate"][-1], str(s["Well"]))
        m.moveTo(x=well['x'], y=well['y'])
        m.dwell(1000)
        dispense()
        count += 1
        if count > 7:
            break
# for frond in fronds:
#     m.moveTo(m.moveTo(x=frond[0] + syringe_off[0], y=frond[1] + syringe_off[1]))
#     aspirate()
#     m.dwell(3000)
#     dispense()
    

Place container of duckweed type **Sp7498** into jubilee and ensure lid is open
Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing
y
G1    E5.00 F1000.00
G1   Z-0.50  F6000.00
G1    E30.00 F1500.00
G1   Z6.00  F500.00
G1    E-30.00 F1400.00


KeyboardInterrupt: 

## Testing Inoculation Loop

In [ ]:
m.toolChange(4)
m.moveTo(x=duckweed_reservoir[0], y=duckweed_reservoir[1])

In [ ]:
# find dip height
collect_height = -34.7
move_height = -9.7
transfer_height = -32.2

In [ ]:
# Inoculation Loop Transfer Test


grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    print("""Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing""")
    input() 
    count = 0
    for index,s in sample_df.iterrows():
        #move to plate and well
        #move to plate and well
        r = 10
        rx = random.randint(-r, r)
        ry = random.randint(-r, r)

        # move in xy first
        m.moveTo(x=duckweed_reservoir[0] + rx, y=duckweed_reservoir[1] + ry)

        # aspirate duckweed
        # first move to surface
        m.moveTo(z=collect_height)
        m.dwell(1000) # wait a bit
        
        m.moveTo(z=move_height)
        well = pp.fetch_well_position(s["Plate"][-1], str(s["Well"]))
        m.moveTo(x=well['x'], y=well['y'])
#         m.dwell(3000)
        m.moveTo(z=transfer_height)
        m.dwell(250)
        m.moveTo(z=move_height)

In [ ]:
# testing RANDOM aspiration

def aspirate():
    m.moveTo(z=duckweed_reservoir[2])
    m.dwell(1000)
    m.move(dz=-0.5) # press slightly
    m.dwell(1000)
    m.move(dz=5, de=50, s=1000) # aspirate!

def dispense():
    m.dwell(1000)
    m.move(de=-40, s=1800)
    m.dwell(500)

grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    print("""Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing""")
    input() 
    count = 0
    for index,s in sample_df.iterrows():
        #move to plate and well
        r = 10
        rx = random.randint(-r, r)
        ry = random.randint(-r, r)

        # move in xy first
        m.moveTo(x=duckweed_reservoir[0] + rx, y=duckweed_reservoir[1] + ry)

        # aspirate duckweed
        # first move to surface
        m.moveTo(z=duckweed_reservoir[2])
        m.dwell(1000) # wait a bit
        m.move(dz=-0.5) # press slightly
        m.dwell(1000)
        m.move(de=20, s=2000) # aspirate!
        m.move(dz=5, s=500) # aspirate!

        m.dwell(1000)
        
        m.moveTo(z=-1.3)
        well = pp.fetch_well_position(s["Plate"][-1], str(s["Well"]))
        m.moveTo(x=well['x'], y=well['y'])
        m.dwell(1000)
        m.move(de=-20, s=2000) # dispense
        count += 1
        # if count > 3:
        #     break
# for frond in fronds:
#     m.moveTo(m.moveTo(x=frond[0] + syringe_off[0], y=frond[1] + syringe_off[1]))
#     aspirate()
#     m.dwell(3000)
#     dispense()
    

In [ ]:
# aspirate duckweed
# M203 E2500 to increase max speed
# M201 E2000 to increase max acc
def aspirate_duckweed(reservoir):
    # move to a random point to pick up duckweed
    # assumes dense petri dish
    r = 25
    rx = r * random.random()
    ry = r * random.random()

    # move in xy first
    m.moveTo(x=reservoir[0] + rx, y=reservoir[1] + ry)
    m.move(de=10, s=1000) # prime the syringe. hopefully not necessary with 10cc.
    # aspirate duckweed
    # first move to surface
    m.moveTo(z=reservoir[2])
    m.dwell(3000)
    m.move(dz=-0.75) # press slightly
    m.dwell(3000)
    m.move(dz=8, de=40, s=2000) # aspirate!
    

In [ ]:
# test aspirate
aspirate_duckweed(duckweed_reservoir)
m.dwell(5000)
m.move(de=-20, s=1500)
#m.move(dz=10, de=-25, s=2100) # aspirate!

In [ ]:
m.dwell(1000)
m.move(dz=-0.5) # press slightly
m.dwell(1000)
m.move(dz=5, de=50, s=1500) # aspirate!

In [ ]:
m.move(de=-40, s=1800)

In [ ]:
#Automated transfer
grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    print("""Type anything into the input field to confirm that the media is available.
    After this point the Jubilee will begin dispensing""")
    input() 
    count = 0
    for index,s in sample_df.iterrows():
        #move to plate and well
        aspirate_duckweed(duckweed_reservoir)
        m.moveTo(z=-42)
        well = pp.fetch_well_position(s["Plate"][-1], str(s["Well"]))
        m.moveTo(x=well['x'], y=well['y'])
        m.move(de=-25, s=2000) # dispense duckweed 
        count += 1
        if count > 3:
            break
        #print("Move to {0}, well {1}".format(s["Plate"], s["Well"]))
        #print("Dispensing media of type {0} into {1}, well {2}".format(field_value,s["Plate"], s["Well"]))


In [ ]:
#Manual transfer

grouped_df = df.groupby('genotype')
for field_value, sample_df in grouped_df:
    print("Place container of duckweed type **{0}** into jubilee and ensure lid is open".format(field_value))
    for index,s in sample_df.iterrows():
        plate = s["Plate"]
        well = s["Well"]
        print(f"Transfer {field_value} to plate {plate}, well {well}")
    input()
        #print("Dispensing media of type {0} into {1}, well {2}".format(field_value,s["Plate"], s["Well"]))


In [ ]:
m.toolChange(3)

In [ ]:
# testing various toolpaths

In [ ]:
m.move(de=20, s=1000) # aspirate!

In [ ]:
m.move(de=-20, s=1000) # dispense!

In [ ]:
# testing showing images with image plates

#Directory that hosts experiment set up file
expt_setup_dir = '/home/pi/duckbot/notebooks/LabDemo/'

#Experimental set up file name, including .json
expt_setup_filename = 'LabDemo.json'

#Host directory that you want to host the folder with your image files. End with forward slash
data_host_dir = '/home/pi/Downloads/'

#Name you want to use for the folder with your image files
data_dir = 'test'
expt_name = 'test' #Will be used as prefix for file names

#Where data will be saved
output_data_dir = data_host_dir + data_dir  

isExist = os.path.exists(output_data_dir)
if not isExist:
  # Create a new directory because it does not exist 
    os.makedirs(output_data_dir)
    print("The new directory is created!")
    
print("Image files will be saved to:")
print(output_data_dir)


In [ ]:
#Drop bed down and add plates
import utils.ImageCaptureUtils as im
print(expt_setup_dir)
print(expt_setup_filename)
plates_to_image = im.pull_plates_to_image(expt_setup_dir, expt_setup_filename)

m.moveTo(z=100)

print(f'Load the following experimental plates into the appropriate positions on the bed: {plates_to_image}')
print("Add plates to the bed. Hit any button to confirm and proceed")

In [ ]:
#Move camera over first well to be imaged
plates_to_image = im.pull_plates_to_image(expt_setup_dir, expt_setup_filename)
im.move_to_first_well(m, plates_to_image)